In [12]:
import numba as nb
import remotenumba as rnb

## Define a function and its signatures

In [13]:
def foo(a, b):
    return a + b

In [18]:
foo_sig = dict(
    ifoo = nb.int64(nb.int64, nb.int64),
    dfoo = nb.float64(nb.float64, nb.float64),
)

## Generate LLVM IR strings

In [19]:
foo_ir = rnb.get_llvm_ir(foo, foo_sig, target='host')

foo_ir_cuda = rnb.get_llvm_ir(foo, foo_sig, target='cuda')

In [23]:
print(foo_ir)

; ModuleID = 'foo'
source_filename = "<ipython-input-13-89a95242c6a7>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@"_ZN08NumbaEnv8__main__8foo$2411Exx" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv8__main__8foo$2412Edd" = common local_unnamed_addr global i8* null

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__8foo$2411Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.16 = add nsw i64 %arg.b, %arg.a
  store i64 %.16, i64* %retptr, align 8
  ret i32 0
}

; Function Attrs: norecurse nounwind readnone
define i64 @ifoo(i64 %.1, i64 %.2) local_unnamed_addr #1 {
entry:
  %.16.i = add nsw i64 %.2, %.1
  ret i64 %.16.i
}

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__8foo$2412Edd"(double* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, double %arg.a, double %arg.

In [ ]:
#print(foo_ir_cuda)

## Compile LLVM IR to machine code - host

In [20]:
eng = rnb.compile(foo_ir)

## Call the functions using cfuncs

In [21]:
from ctypes import CFUNCTYPE, c_double, c_int
ifoo = CFUNCTYPE(c_int, c_int, c_int)(eng.get_function_address('ifoo'))
dfoo = CFUNCTYPE(c_double, c_double, c_double)(eng.get_function_address('dfoo'))

In [22]:
ifoo(1,2), dfoo(3.4, 2)

(3, 5.4)